In [ ]:
video_capture = cv2.VideoCapture(0)

folder_path = 'C:/Users/Mahin/Desktop/Image_Project/Facial Recognition Attendence/Image'


image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png', '.jpeg'))]

# Initialize an empty list to store the face encodings
known_face_encoding = []

# Encode each image in the list of images
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = fc.load_image_file(image_path)
    encoding = fc.face_encodings(image)
    
    # If a face is detected and encoded, add it to the list of encodings
    if len(encoding) > 0:
        known_face_encoding.append(encoding[0])

known_face_name = ["Mahin", "Mofazzal", "Rashed", "Rony", "Uttam"]

students = known_face_name.copy()
face_location = []
face_encodings = []
face_names = []
unknown_face_encodings = []

# Timeout parameters (Adjust these values as needed)
timeout_duration = 30  # Number of seconds for timeout
last_face_detected_time = datetime.now()

# Motion detection parameters (Adjust these values as needed)
motion_threshold = 10000  # Adjust this threshold based on your camera and environment
previous_gray_frame = None # Initialize the variable before the loop

s = True

now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
f = open(current_date+'.csv','w+',newline= '')
lnwriter = csv.writer(f)

while True :
    _,frame = video_capture.read()
    small_frame = cv2.resize(frame,(0,0),fx = 0.25,fy = 0.25)
    rgb_small_frame = small_frame[:,:,::-1]
     # Timeout mechanism
    if len(unknown_face_encodings) == 0:
        current_time = datetime.now()
        elapsed_time = (current_time - last_face_detected_time).seconds
        if elapsed_time > timeout_duration:
            print("Timeout reached. No faces detected in the last {} seconds.".format(timeout_duration))
            break
    if s:
        # store the data which is coming from the webcame
        face_location = fc.face_locations(rgb_small_frame)
        face_encodings = fc.face_encodings(rgb_small_frame,face_location)
        # finding and compare the name
        face_names = []
        for face_encoding in face_encodings :
            if len(known_face_encoding) > 0: 
                matches = fc.compare_faces(known_face_encoding, face_encoding)
                name = "Unknown"
                face_distance = fc.face_distance(known_face_encoding, face_encoding)
                if np.any(face_distance < 0.6): 
                    best_match_index = np.argmin(face_distance)
                    if matches[best_match_index]:
                         name = known_face_name[best_match_index]
                    
                face_names.append(name)
                # Append unknown faces to the unknown_face_encodings list
                if name == "Unknown":
                    unknown_face_encodings.append(face_encoding)
                    last_face_detected_time = datetime.now()   # Update last face detected time
        # inserting CSV file
        for name in face_names:
            if name in known_face_name:
                 if name in students:
                        students.remove(name)
                        print(students)
                        current_time = datetime.now()
                        current_time_str = current_time.strftime("%H-%M-%S")
                        lnwriter.writerow([name, current_time_str])
    # Motion detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)
    
    if len(unknown_face_encodings) == 0:
        motion_detected = False
        if previous_gray_frame is not None:
            frame_delta = cv2.absdiff(previous_gray_frame, gray_frame)
            thresh = cv2.threshold(frame_delta, motion_threshold, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.dilate(thresh, None, iterations=2)
            contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                for contour in contours:
                    motion_detected = True
                    break
        if motion_detected:
            print("Motion detected.")
            last_face_detected_time = datetime.now()  # Reset last face detected time
    previous_gray_frame = gray_frame
    
    cv2.imshow("attendence system", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break

# show output
video_capture.release()
cv2.destroyAllWindows()
f.close()